In [41]:
import pandas as pd
df = pd.read_csv('Data/campeonato_brasileiro_1971_1976.csv')
df

,Unnamed: 0,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
0,0,Série A,1971,1971-08-07,Santa Cruz,1,4,Corinthians,VV,0,3,1
1,9,Série A,1971,1971-08-07,Botafogo,0,0,America-RJ,E,1,1,10
2,7,Série A,1971,1971-08-07,Bahia,0,0,Santos,E,1,1,8
3,6,Série A,1971,1971-08-07,Sport,1,0,Flamengo,VC,3,0,7
4,5,Série A,1971,1971-08-07,São Paulo,0,3,Grêmio,VV,0,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...
2506,404,Série A,1976,1976-11-28,Guarani,2,3,Vasco,VV,0,3,405
2507,406,Série A,1976,1976-11-28,Atlético-MG,0,0,Grêmio,E,1,1,407
2508,408,Série A,1976,1976-12-05,Internacional,2,1,Atlético-MG,VC,3,0,409
2509,407,Série A,1976,1976-12-05,Fluminense,1,1,Corinthians,E,1,1,408


In [42]:
teams = ['Corinthians', 'São Paulo', 'Santos', 'Palmeiras']

classicos = ((df['Casa'].isin(['Corinthians', 'Palmeiras', 'São Paulo', 'Santos']))
              & (df['Fora'].isin(['Corinthians', 'Palmeiras', 'São Paulo', 'Santos'])))
classicos = df[classicos]

derby = ((df['Casa'].isin(['Corinthians', 'Palmeiras']))
              & (df['Fora'].isin(['Corinthians', 'Palmeiras'])))
derby = df[derby]

majestoso = ((df['Casa'].isin(['Corinthians', 'São Paulo']))
              & (df['Fora'].isin(['Corinthians', 'São Paulo'])))
majestoso = df[majestoso]

alvinegro = ((df['Casa'].isin(['Corinthians', 'Santos']))
              & (df['Fora'].isin(['Corinthians', 'Santos'])))
alvinegro = df[alvinegro]

choque_rei = ((df['Casa'].isin(['Palmeiras', 'São Paulo']))
              & (df['Fora'].isin(['Palmeiras', 'São Paulo'])))
choque_rei = df[choque_rei]

pal_x_san = ((df['Casa'].isin(['Palmeiras', 'Santos']))
              & (df['Fora'].isin(['Palmeiras', 'Santos'])))
pal_x_san = df[pal_x_san]

sansao = ((df['Casa'].isin(['São Paulo', 'Santos']))
              & (df['Fora'].isin(['São Paulo', 'Santos'])))
sansao = df[sansao]


In [46]:
classicos_stats

,Time,Partidas,Vitórias,Derrotas,Empates,Gols,Pontos,Percentual de Vitórias,Percentual de Empates,Percentual de Derrotas,Percentual de Pontos
1,São Paulo,22,7,7,8,18,22,31.818182,36.363636,31.818182,50.000000
0,Santos,16,8,3,5,21,21,50.000000,31.250000,18.750000,65.625000
2,Corinthians,20,5,5,10,11,20,25.000000,50.000000,25.000000,50.000000
3,Palmeiras,22,3,8,11,9,17,13.636364,50.000000,36.363636,38.636364


In [47]:
import pandas as pd

def update_team_stats(df):
    team_stats = {}

    for index, row in df.iterrows():
        home_team, away_team = row['Casa'], row['Fora']
        result = row['Res']

        # Update home team stats
        team_stats.setdefault(home_team, {'Partidas': 0, 'Vitórias': 0, 'Derrotas': 0, 'Empates': 0, 'Gols': 0, 'Pontos': 0})
        team_stats[home_team]['Partidas'] += 1
        team_stats[home_team]['Gols'] += row['GC']

        if result == 'VC':
            team_stats[home_team]['Vitórias'] += 1
            team_stats[home_team]['Pontos'] += 2
        elif result == 'VV':
            team_stats[home_team]['Derrotas'] += 1
        else:
            team_stats[home_team]['Empates'] += 1
            team_stats[home_team]['Pontos'] += 1

        # Update away team stats
        team_stats.setdefault(away_team, {'Partidas': 0, 'Vitórias': 0, 'Derrotas': 0, 'Empates': 0, 'Gols': 0, 'Pontos': 0})
        team_stats[away_team]['Partidas'] += 1
        team_stats[away_team]['Gols'] += row['GF']

        if result == 'VC':
            team_stats[away_team]['Derrotas'] += 1
        elif result == 'VV':
            team_stats[away_team]['Vitórias'] += 1
            team_stats[away_team]['Pontos'] += 2
        else:
            team_stats[away_team]['Empates'] += 1
            team_stats[away_team]['Pontos'] += 1

    # Create a DataFrame with team statistics
    team_stats_df = pd.DataFrame.from_dict(team_stats, orient='index').reset_index()
    team_stats_df = team_stats_df.rename(columns={'index': 'Time'})
    team_stats_df = team_stats_df.sort_values(by='Pontos', ascending=False)

    # Calculating the percentage of victories, draws, and losses
    team_stats_df['Percentual de Vitórias'] = (team_stats_df['Vitórias'] / team_stats_df['Partidas']) * 100
    team_stats_df['Percentual de Empates'] = (team_stats_df['Empates'] / team_stats_df['Partidas']) * 100
    team_stats_df['Percentual de Derrotas'] = (team_stats_df['Derrotas'] / team_stats_df['Partidas']) * 100

    # Calculating the percentage of total points based on the maximum possible points
    max_possible_points = team_stats_df['Partidas'] * 2
    team_stats_df['Apoveitamento'] = (team_stats_df['Pontos'] / max_possible_points) * 100

    return team_stats_df

# Example usage:
classicos_stats = update_team_stats(classicos)
derby_stats = update_team_stats(derby)
majestoso_stats = update_team_stats(majestoso)
alvinegro_stats = update_team_stats(alvinegro)
choque_rei_stats = update_team_stats(choque_rei)
pal_x_san_stats = update_team_stats(pal_x_san)
sansao_stats = update_team_stats(sansao)
print(classicos_stats)
print(derby_stats)
print(majestoso_stats)
print(choque_rei_stats)
print(pal_x_san_stats)
print(sansao_stats)

          Time  Partidas  Vitórias  Derrotas  Empates  Gols  Pontos  \
1    São Paulo        22         7         7        8    18      22   
0       Santos        16         8         3        5    21      21   
2  Corinthians        20         5         5       10    11      20   
3    Palmeiras        22         3         8       11     9      17   

   Percentual de Vitórias  Percentual de Empates  Percentual de Derrotas  \
1               31.818182              36.363636               31.818182   
0               50.000000              31.250000               18.750000   
2               25.000000              50.000000               25.000000   
3               13.636364              50.000000               36.363636   

   Apoveitamento  
1      50.000000  
0      65.625000  
2      50.000000  
3      38.636364  
          Time  Partidas  Vitórias  Derrotas  Empates  Gols  Pontos  \
0  Corinthians         8         2         1        5     4       9   
1    Palmeiras         8  

In [44]:
classicos_stats


,Time,Partidas,Vitórias,Derrotas,Empates,Gols,Pontos,Percentual de Vitórias,Percentual de Empates,Percentual de Derrotas,Percentual de Pontos
1,São Paulo,22,7,7,8,18,22,31.818182,36.363636,31.818182,50.000000
0,Santos,16,8,3,5,21,21,50.000000,31.250000,18.750000,65.625000
2,Corinthians,20,5,5,10,11,20,25.000000,50.000000,25.000000,50.000000
3,Palmeiras,22,3,8,11,9,17,13.636364,50.000000,36.363636,38.636364


In [14]:
teams = majestoso_stats['Team']
victories = majestoso_stats['Victories']
draws = majestoso_stats['Draws']
losses = majestoso_stats['Losses']

# Plotting a bar chart
fig, ax = plt.subplots(figsize=(10, 6))

bar_width = 0.2
index = range(len(teams))

bar1 = ax.bar(index, victories, bar_width, label='Victories')
bar2 = ax.bar([i + bar_width for i in index], draws, bar_width, label='Draws')
bar3 = ax.bar([i + 2 * bar_width for i in index], losses, bar_width, label='Losses')

# Adding labels, title, and legend
ax.set_xlabel('Teams')
ax.set_ylabel('Number of Games')
ax.set_title('Number of Games by Result')
ax.set_xticks([i + bar_width for i in index])
ax.set_xticklabels(teams, rotation=45, ha='right')
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()

NameError: name 'majestoso_stats' is not defined